In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

2024-08-05 18:51:16.289165: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 18:51:16.310782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 18:51:16.334390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 18:51:16.340402: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 18:51:16.358315: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data = pd.read_csv('/home/jeanluca/code/JeanLucaSchindler/Portfolio/Sentiment_Analysis/Sentiment-Analysis-for-Financial-News/raw_data/all-data.csv', encoding='ISO-8859-1', header=None)

In [3]:
data = data.rename(columns={0:'sentiment', 1:'text'})

In [4]:
labels = {'negative':0,
          'neutral':1,
          'positive':2
          }

def transform_labels(label):
    return labels[label]

In [5]:
data['label'] = data.sentiment.apply(transform_labels)

In [88]:
data.text.iloc[7]

'In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .'

In [7]:
y = data[['label']]

X = data[['text']]

In [23]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(X.to_numpy(),
                                                                            y.to_numpy(),
                                                                            test_size = 0.2,
                                                                            random_state = 42)

In [26]:
# Flattening to ensure that each element is a string
train_sentences = [sentence[0] for sentence in train_sentences]
val_sentences = [sentence[0] for sentence in val_sentences]

# Ensure labels are in the correct shape
train_labels = train_labels.flatten()
val_labels = val_labels.flatten()


In [42]:
# For multi-class classification setup
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as hub

class USELayerWithExpandDims(Layer):
    def __init__(self, **kwargs):
        super(USELayerWithExpandDims, self).__init__(**kwargs)
        self.use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

    def call(self, inputs):
        use_output = self.use(inputs)
        return tf.expand_dims(use_output, axis=1)

inputs = layers.Input(shape=(), dtype=tf.string, name="input_layer")
x = USELayerWithExpandDims()(inputs)
x = layers.Bidirectional(layers.LSTM(72, return_sequences=True))(x)
x = layers.Dropout(0.5)(x)
x = layers.Bidirectional(layers.LSTM(72, return_sequences=True))(x)
x = layers.Dropout(0.5)(x)
x = layers.Bidirectional(layers.LSTM(72))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation='softmax', name='output_layer')(x)

model = tf.keras.Model(inputs, outputs, name="model_lstm")
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

# Ensure labels are one-hot encoded
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=3)
val_labels = tf.keras.utils.to_categorical(val_labels, num_classes=3)

# Convert datasets to include one-hot encoded labels
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels)).batch(32).prefetch(tf.data.AUTOTUNE)
valid_dataset = tf.data.Dataset.from_tensor_slices((val_sentences, val_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [43]:
history = model.fit(train_dataset, validation_data = valid_dataset, epochs = 10)

Epoch 1/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 132s 780ms/step - accuracy: 0.5826 - loss: 0.9802 - val_accuracy: 0.6649 - val_loss: 0.7014
Epoch 2/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 75s 615ms/step - accuracy: 0.6631 - loss: 0.7061 - val_accuracy: 0.7381 - val_loss: 0.6307
Epoch 3/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 79s 647ms/step - accuracy: 0.7295 - loss: 0.6289 - val_accuracy: 0.7557 - val_loss: 0.5743
Epoch 4/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 80s 658ms/step - accuracy: 0.7707 - loss: 0.5531 - val_accuracy: 0.7680 - val_loss: 0.5423
Epoch 5/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 79s 636ms/step - accuracy: 0.7896 - loss: 0.5088 - val_accuracy: 0.7598 - val_loss: 0.5510
Epoch 6/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 76s 625ms/step - accuracy: 0.7955 - loss: 0.4896 - val_accuracy: 0.7660 - val_loss: 0.5461
Epoch 7/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 77s 634ms/step - accuracy: 0.7998 - loss: 0.4772 - val_accuracy: 0.7680 - val_loss: 0.5464
Epoch 8/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 78s 643ms/step - accuracy: 0.8072 - loss: 

In [49]:
history

In [52]:
y_probs = model.predict(valid_dataset)

31/31 ━━━━━━━━━━━━━━━━━━━━ 27s 781ms/step


In [53]:
y_preds = tf.round(y_probs)

In [54]:
y_preds[:10]

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)>

In [56]:
model.evaluate(valid_dataset)

31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 280ms/step - accuracy: 0.7807 - loss: 0.5482


[0.5454019904136658, 0.7711340188980103]

In [57]:
# model.save('best_model.h5')

In [59]:
# from tensorflow.keras.models import load_model

# # Define the custom layer class again
# class USELayerWithExpandDims(tf.keras.layers.Layer):
#     def __init__(self, **kwargs):
#         super(USELayerWithExpandDims, self).__init__(**kwargs)
#         self.use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

#     def call(self, inputs):
#         use_output = self.use(inputs)
#         return tf.expand_dims(use_output, axis=1)

# # Load the model, ensuring the custom layer is registered
# model = load_model('/home/jeanluca/code/JeanLucaSchindler/Portfolio/Sentiment_Analysis/Sentiment-Analysis-for-Financial-News/best_model.h5', custom_objects={'USELayerWithExpandDims': USELayerWithExpandDims})


In [102]:
custom = [
    # Positive News
    "Tech company reports record profits for the second quarter in a row.",
    "New partnership announcement boosts company's stock price by 15%.",
    "Startup secures major investment, plans to expand globally.",
    "The new product launch has been a tremendous success, receiving rave reviews.",

    # Negative News
    "Shares plummet as the company faces allegations of financial misconduct.",
    "Company announces major layoffs due to declining sales.",
    "The market reacts negatively to the disappointing earnings report.",
    "Product recall issued due to safety concerns.",

    # Neutral News
    "The CEO will present the company's new strategy at the upcoming conference.",
    "The annual shareholder meeting is scheduled for next month.",
    "Company releases its new range of eco-friendly products.",
    "Board of directors announces the appointment of a new CFO.",

    # Mixed or Ambiguous Sentiments
    "While the new product received mixed reviews, initial sales figures are strong.",
    "The company's revenue grew, but profit margins are shrinking due to rising costs.",
    "Investors are cautious after the merger announcement, with some expecting market volatility.",
    "Despite the strong market position, the company's future remains uncertain due to regulatory challenges."
]


In [103]:
test = tf.data.Dataset.from_tensor_slices((custom, None)).batch(32).prefetch(tf.data.AUTOTUNE)

In [107]:
pred = model.predict(test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step


In [121]:
preds = np.round(pred, 2)
preds

array([[0.08, 0.46, 0.46],
       [0.  , 0.46, 0.54],
       [0.  , 0.41, 0.59],
       [0.  , 0.03, 0.97],
       [0.84, 0.13, 0.02],
       [0.85, 0.06, 0.09],
       [0.88, 0.04, 0.08],
       [0.63, 0.3 , 0.07],
       [0.  , 0.98, 0.02],
       [0.  , 0.99, 0.01],
       [0.  , 0.72, 0.28],
       [0.  , 0.94, 0.06],
       [0.01, 0.15, 0.84],
       [0.55, 0.03, 0.42],
       [0.07, 0.9 , 0.03],
       [0.86, 0.09, 0.06]], dtype=float32)